# Flask API, CNN Project, Deploy on website
model is loaded from weights trained from an external Keras Sequential CNN.<br>
Flask returns image and results after classifying image
* Date: 02/27/2021
* Author: Don Agiro

# Setup & gpu Activation
NB: you can comment out GPU section if not needed

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.compat.v1.keras.backend import set_session

import numpy as np
import pandas as pd

# comment out section below if you do not have a GPU supported device
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1



## Load Model
load your model using keras models. <br>
you can use pickle, dill etc or any other package you initially used to save weights.

In [2]:
loaded_model = tf.keras.models.load_model(
    "./mosa", 
    custom_objects=None, 
    compile=True, options=None
)

## Define function for Image
this function will be used to `preprocess` images and use loaded weights to classify any images submitted for analysis<br>
you can change this part to however you want your images analysed. should work for all binary type clasifications 

In [3]:
def pred(x):
    # set constants, use the dimensions as defined in your original model
    image_size = (52, 52)
    
    # load image file using keras preprocessing
    img = keras.preprocessing.image.load_img(x, target_size=image_size)

    # very low level preprocessing
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # read documentation for "expand_dims" on keras

    # predict the images class using loaded weights.
    predictions = loaded_model.predict(img_array)
    score = predictions[0]
    
    # use this conditional statement if you to return human readable output 
    # not a numerical value.
    # else you can comment it out and return score
    if score > 0.5:
        result = "Happy :)"
    else:
        result = "Sad :("

    return result

In [4]:
# use an image to test function above before deploying publicly
test_img = "./input/test/sad/PrivateTest_6048665.jpg"
img_class = pred(test_img)
img_class

'Sad :('

# Flask

Deploy application to a website using flask

In [ ]:
import os
import imghdr
from werkzeug.utils import secure_filename
from flask import Flask, render_template, request, redirect, url_for, abort, \
    send_from_directory

In [5]:
# the folder holding uploaded images is photo which is a subfolder of static
POLDER = os.path.join('static', 'photo')

# define flask constants 
app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 1024 * 1024
app.config['UPLOAD_EXTENSIONS'] = ['.jpg', '.png', '.jpeg']
app.config['UPLOAD_PATH'] = POLDER

# this will be used later to empty upload folder before any new analysis
mydir = POLDER

# validate user uploaded right image type/extension
def validate_image(stream):
    header = stream.read(512)  # 512 bytes should be enough for a header check
    stream.seek(0)  # reset stream pointer
    format = imghdr.what(None, header)
    if not format:
        return None
    return '.' + (format if format != 'jpeg' else 'jpg')

@app.route('/')
def index():
    files = os.listdir(app.config['UPLOAD_PATH'])
    return render_template('index.html', files=files)

@app.route('/', methods=['POST'])
def upload_files():
    
    # loop function to empty upload folder before adding new file
    for f in os.listdir(mydir):
        if not f.endswith(".jpg"):
            continue
        os.remove(os.path.join(mydir, f))
    
    # now we start the process of uploading image to defined upload foldder
    uploaded_file = request.files['file']
    filename = secure_filename(uploaded_file.filename)
    if filename != '':
        file_ext = os.path.splitext(filename)[1]
        if file_ext not in app.config['UPLOAD_EXTENSIONS'] or \
                file_ext != validate_image(uploaded_file.stream):
            abort(400)
        uploaded_file.save(os.path.join(app.config['UPLOAD_PATH'], filename))
        
        test_img = os.path.join(app.config['UPLOAD_PATH'], filename)
        img_class = pred(test_img)
        results = img_class
        
    # generates and saves image file path to be retrieved for display later when sending analysis results
    full_filename = os.path.join(app.config['UPLOAD_PATH'], filename)
    
    # return analysis results and image that was analysed
    return render_template("index.html", model_result = results, user_image = full_filename)

@app.route('/uploads/<filename>')
def upload(filename):
    return send_from_directory(app.config['UPLOAD_PATH'], filename)

In [8]:
if __name__ == '__main__':
    app.run(debug=True, use_debugger=False, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Feb/2021 12:18:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2021 12:18:42] "POST / HTTP/1.1" 200 -
